<img src='https://github.com/LinkedEarth/Logos/raw/master/PYLEOCLIM_logo_HORZ-01.png' width="800">

# 3. Exploratory analysis of a LiPD dataset  

If all you have is a single record, you can get away with Excel. But most interesting work nowadays involves integrating several records together, and this a where a package like `pyleoclim` shines. 
The most elementary level of data analysis in paleoclimatology is to visualize the timeseries themselves, and see how they line up with other records. This notebook shows a few nifty tricks on how to do that without breaking a sweat. It also introduces various functionalities like:
- slicing
- interpolation
- binning
- coarse-graining
- standardization
- singular spectrum analysis
- wavelet coherence

We will use the dataset of [Simon et al (2020)](https://www.nature.com/articles/s41597-020-00689-7), which contains many data streams pertaining to the variability of the Algulhas current over the past 270,000 years. 

## Loading packages and data

Pyleoclim has several visual styles. So far, we have been using the default style `journal`, which is perfect to create publication-ready visualizations. However, if your goal is to share a notebook or embed a graphic on a webpage, you might want to use `web`. A list of Pyleoclim styles is available [here](https://pyleoclim-util.readthedocs.io/en/latest/utils/plotting/set_style.html#utils-set-style).

In [ ]:
import pyleoclim as pyleo
import numpy as np
pyleo.set_style('web')  # set the visual style

Let's load the dataset into a `Lipd` object.

In [ ]:
d = pyleo.Lipd('../data/Agulhas Current_CD154_10-06P.Simon.2020.lpd')

Let's look at what's in the file:

In [ ]:
ts_list = d.to_tso() 
for idx, item in enumerate(ts_list):
    print(str(idx)+': '+item['dataSetName']+': '+item['paleoData_variableName'])

Many of these series correspond to different epochs: some concern the recent past, some the whole 270,000y time span.
Let's skip those series dealing with age and depth, and put the rest into a proper _MultipleSeries_ objects to unlock `pyleoclim` magic.

In [ ]:
ts_slice = ts_list[3:13]+ts_list[15:] # manual selection. Sometimes, you have to!
tsl=[]
for idx, item in enumerate(ts_slice):
    ts = pyleo.LipdSeries(item)  # extract the Series
    tsl.append(ts) # add it to the list
    tsl[idx].label = ts.lipd_ts['paleoData_variableName']  # tweak the label
ms = pyleo.MultipleSeries(tsl,name='Simon et al [2020] dataset', time_unit = 'ky BP') # create MultipleSeries object from this list

One way to peek at the object's contents is to invoke this method:

In [ ]:
ms.__dict__

From here on out, there are various ways to explore. The first one is to just plot the series as they are:

In [ ]:
fig, ax = ms.plot(lgd_kwargs={'bbox_to_anchor': [1.1, 1]}) 

Notice that we needed to tweak the arguments of the legend (`lgd_kwargs`) a trifle so that it wouldn't cover the actual traces.  Now, these series correspond to several types of measurements expressed in different units. For a more meanignful comparison, one might wish to standardize them first. Yes, there is a button for that:

In [ ]:
fig, ax = ms.standardize().plot(lgd_kwargs={'bbox_to_anchor': [1.1, 1]})

**Exercise 3.1**: customize the plot by:
 - tweaking the color scheme for the various traces (read the doc of `ms.plot()` to figure out how)
 - tweaking the plot style (see [this tutorial](https://github.com/LinkedEarth/Pyleoclim_util/blob/master/example_notebooks/plot_styles.ipynb)).
 - adding a title and a more descriptive label on the y axis


In [ ]:
pyleo.set_style('journal')
ms.standardize().plot(lgd_kwargs={'bbox_to_anchor': [1.1, 1]})

Another available plot type is a _stackplot_:

In [ ]:
fig, ax = ms.stackplot()

If the writing is too obstrusive and the lines too thick, you can lighten up and space out things thusly:

In [ ]:
ms.stackplot(figsize=[5, 10], font_scale=0.4, linewidth = 0.8)

And if you wanted to tweak the color theme, this is how you would do it:

In [ ]:
import matplotlib.colors as colors
norm = colors.Normalize(vmin=0, vmax=16)
fig, ax = ms.stackplot(figsize=[5, 10], font_scale=0.6, linewidth = 0.8, cmap='twilight',norm=norm)

(if you don't like `twilight`, substitute any [Matplotlib colormap](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html) you like.)

Back to the dataset: it contains several reconstructions, some provided with uncertainties. Let us plot three relevant groups:

### Mg/Ca SST

Trace metal ratios in planktonic foraminifera should hold no secrets for you. If they do, this is not where you'll learn about them. Several estimates of SST derived from Mg/Ca are provided in this dataset:
The original Mg/Ca data, in accordance with the [PaCTS](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019PA003632) standard, but also two different SST reconstructions derived from it, using two paleotemperature equations: [Anand et al (2003)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2002PA000846), and [Gray & Evans (2019)](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/2018PA003517). The latter comes with error estimates that were archived in this LiPD file, so we plot them below. First Anand et al 2003:

In [ ]:
ts = ms.series_list[2];
fig, ax = ts.plot()

In [ ]:
fig, ax = ts.plot()
ts2 = ms.series_list[3]; 
ts3 = ms.series_list[4]; gray2019_uncert = ts3.value; age = ts3.time 
ax.fill_between(age,ts2.value-2*gray2019_uncert,ts2.value+2*gray2019_uncert,color='C1',alpha=0.3,label=r'$\pm 2\sigma$')
ts2.plot(ax=ax) # plot the Gray et al SST over the uncetainty band, in the same axes
ax.legend()
ax.set_ylabel(u'SST (\N{DEGREE SIGN} C)') # let's give this some proper units. 
ax.set_title('Mg/Ca based SST reconstructions')

We see that the Gray (2019) equation leads to temperatures that tend to be slightly cooler, especially at the Last Glacial Maximum and in the Late Holocene. This difference does not appear very meaningful given the large uncertainties, but just for shits and giggles, this is how you would compute it, using the `slice` feature: 

In [ ]:
mgcasst = pyleo.MultipleSeries([ts, ts2]) # create multiple series object
deltaT = [None]*2
for i, tz in enumerate(mgcasst.series_list):
    modern = tz.slice([0, 3]).value.mean()
    lgm    = tz.slice([18, 21]).value.mean()
    deltaT[i] = modern - lgm
    print(f"LGM-modern cooling in {tz.label} is {deltaT[i]:.2f} \N{DEGREE SIGN}C")

So a 0.5K difference, well within uncertainties. 

**Exercise 3.2a** Repeat this exercise to compare the early and late Holocene.

In [ ]:
## Your code here ##

**Exercise 3.2b** Use these results to more rigorously assess whether the difference in estimated LGM-modern warming is statistically significant (hint: use [T-test from SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind_from_stats.html)). You will have to get your hands dirty with the data, but dirty hands are learning hands!

In [ ]:
## Your code here ##

### Uk37 and TEX86 SST

Another group of proxies taken on different sensors (alkenones and archea) are available too: Uk'37 and TEX86-based SST reconstructions:

In [ ]:
ts = ms.series_list[6]
fig, ax = ts.plot(label='Uk37 SST')
d2 = ts_list[15]; sst = pyleo.LipdSeries(d2);  age  = sst.time
ts = pyleo.LipdSeries(ts_list[16]); tiern2016_lower = ts.value; 
ts = pyleo.LipdSeries(ts_list[17]); tiern2016_upper = ts.value
ax.fill_between(age,tiern2016_lower,tiern2016_upper,color='C1',alpha=0.3,label=r'$\pm 2\sigma$')
sst.plot(ax=ax,label='TEX86')
ax.legend()
ax.set_ylabel(u"SST (\N{DEGREE SIGN} C)")

ax.set_title('Uk37 and TEX86 based SST reconstructions')

### Seawater d18O

The dataset also provides estimates of seawater $\delta^{18}O$, a proxy for salinity:

In [ ]:
d18Osw = ms.series_list[13]; age  = d18Osw.time; 
fig, ax = d18Osw.plot(label='median')
d18Osw_lower = ms.series_list[14]; d18Osw_upper = ms.series_list[15]
ax.fill_between(age,d18Osw_lower.value,d18Osw_upper.value,color='C0',alpha=0.3,label=r'$\pm 2\sigma$')
ax.legend()
ax.set_ylabel(r'$\delta^{18}O_{sw}$ ' + u'(\u2030)')
ax.set_title('Seawater oxygen isotopic composition')

**Exercise 3.3**: add a dashboard of this record to get its essential features at a glance, leaving out metadata.

In [ ]:
## Your code here ##

One thing that jumps out from the last record is the "pull of the present": because of sediment compaction and coring methods, the last 30,000 years or so have a much higher resolution than the 240,000 years before that. If we are going to compare similar intervals (e.g. deglaciations), it would be unwise to do so on unprocessed data. To be sure that we are comparing climate features, we must remove (or at least minimize) taphonomic effects like compaction. 

## Coarse-graining
Pyleoclim offers 3 ways to bring the timeseries to a uniform spacing: interpolation, binning, and coarse-graining via a Gaussian kernel, as used in [Rehfeld et al. (2011)](https://doi.org/10.5194/npg-18-389-2011). Let's start with interpolation with default parameters:

In [ ]:
fig, ax = d18Osw.plot(label='median')
d18Oswi = d18Osw.interp()
d18Oswi.plot(ax=ax,label='linear interpolation')
ax.fill_between(age,d18Osw_lower.value,d18Osw_upper.value,color='C0',alpha=0.3,label=r'$\pm 2\sigma$')
ax.legend()
ax.set_ylabel(r'$\delta^{18}O_{sw}$ ' + u'(\u2030)')
ax.set_title('Seawater oxygen isotopic composition')

**Exercise 3.4**: look up the `interp` method to find how to implement a cubic spline interpolation and plot the result on the same figure as above. Comment on the difference(s). 

In [ ]:
## your work here ##

The second method is binning, which by default is conservative:

In [ ]:
fig, ax = d18Osw.plot()
d18Oswb = d18Osw.bin()
d18Oswb.plot(ax=ax,label='binning [default]')

**Exercise 3.5**: change the bin size to 5ky. What do you observe?

In [ ]:
fig, ax = d18Osw.plot()
d18Oswb.plot(ax=ax,label='binning [default]')
d18Oswb = d18Osw.bin(bin_size=5)
d18Oswb.plot(ax=ax,label='binning [5ky]')

The last method uses a Gaussian kernel. Here it is in action:

In [ ]:
fig, ax = d18Osw.plot()
d18Oswk = d18Osw.gkernel()
d18Oswk.plot(ax=ax,label='Gaussian kernel, default')

You may want to play with `step_style` and the kernel bandwidth `h` a bit.


**Exercise 3.5**: Compare the 3 methods on the same plot. Which method are you most likely to use and why (it's OK if all team members have different opinions; just make sure you justify your answers)


## Singular spectrum analysis
One way to extract some features of interest is with Singular Spectrum Analysis or SSA (see e.g. [this tutorial](https://nbviewer.jupyter.org/github/LinkedEarth/LiPDbooks/blob/master/notebooks/singular_spectrum_analysis.ipynb)). Like most timeseries methods, SSA assumes uniform time spacing, for which we use the linearly interpolated version, `d18Oswi`. 

In [ ]:
ssa  = d18Oswi.ssa(trunc='var')

### SSA approximation

Let's first examine the eigenvalue spectrum, which may be interpreted (as in principal component analysis) in terms of the fraction of variance accounted for by each mode:

In [ ]:
ssa.screeplot()

As is often the case with geophysical timeseries, we see a very red spectrum, with the first 5 modes (0 to 4) accounting for the vast majority of the variance. We can use this property to obtain a low-order approximation of the original data using those first few modes:

In [ ]:
d18OswiSSA = d18Oswi.copy()
d18OswiSSA.value = ssa.RCseries
f2, ax = d18Osw.plot(label='original')
d18Oswi.plot(ax=ax,label='interpolated')
d18OswiSSA.plot(label='SSA approximation',ax=ax,color='Purple')

**Exercise 3.6**: 
- play with the window size (parameter M) to check the robustness of the results.
- use the truncation feature of the SSA method; check the effect of the 3 truncation methods and their optional parameters.

In [ ]:
## your work here ##

### SSA interpolation

The SSA algorithm implemented in `pyleoclim` is tolerant of missing values. If the only reason to put the data on a uniform time grid was to apply SSA, then we could bypass that step altogether. 

**Exercise 3.7**: 
- use the `bin` method with a bin size of 2.5 to generate an evenly spaced time axis $t$ and incomplete series of values $y(t)$. 
- apply ssa() to this series and plot the result
- compare this solution to the SSA approximation derived above from interpolated data, and to the interpolated data on their own. Which solution do you prefer and why?

In [ ]:
## your work here ##

## Wavelet coherency 

Another feature of pyleoclim is that it makes it quite easy to perform very complex workflows in a few keystrokes. Here we illustrate how to perform [wavelet transform coherency](https://hal.archives-ouvertes.fr/hal-00302394/) between the $\delta^{18}O_{sw}$ and TEX86-based SST. 

In [ ]:
coh = sst.wavelet_coherence(d18Osw) # compute coherence

In [ ]:
fig, ax = coh.plot()  # plot it

**Exercise 3.8** Interpret the figure: what is the meaning of the arrows, colors and dashed lines? On the whole, is variability SST and d18Osw coherent? If so, at what level of confidence? (hint: type `coh.signif_test?` to implement a significance test. Warning: it make take a few minutes to complete, but such is the price of rigor!)

In [ ]:
## your work here ##

The "islands" outlined in white are deemed significant according to this test.

**Exercise 3.9** compare the d18Osw series to the [benthic d18O stack](https://academic.oup.com/climatesystem/article/2/1/dzx002/3892410) (proxy for ice volume) or another series of your choice covering a comparable time interval.  The ProbStack data are available through:

In [ ]:
import xarray as xr

ds = xr.open_dataset('../data/PNAS19_data.nc')
ds

ts, vs = {}, {}

for name in ds.variables:
    if name != 'time':
        ts[name] = ds[name].time
        vs[name] = ds[name].values

#print(vs.keys())
ProbStack = pyleo.Series(
        time=-ts['ProbStack']/1000,
        value=vs['ProbStack'],
        label='ProbStack',
        time_name='Time',
        time_unit='kyrs',
        value_name=r'$\delta^{18}$O',
        value_unit=u'\u2030')

In [ ]:
## your code here ##

Note: if ice is more your type, the Epica Dome C record is under `ts['EDC']` and `vs['EDC']` and could be a fun series to compare the d18Osw to. 

## Multivariate Recreations

Our least bit of fun with this dataset is to identify a common signal between various temperature reconstructions. This is one of the many things one can do with Principal Component Analysis (PCA). First, we extract 5 median estimates of SST from the wider dataset:

In [ ]:
for i, ts in enumerate(ms.series_list):
    print(i, ts.label)

In [ ]:
indices = [3,6,8,10] # indices of  temperature datasets
tsl=[]
for i in indices:
    tsl.append(ms.series_list[i])
mst = pyleo.MultipleSeries(tsl,name='S20 temperature', time_unit = 'ky BP')

In [ ]:
mst.stackplot()

Let's put them all on a common timescale using gkernel and a step size of 500y, and standardize them to remove offsets:

In [ ]:
ms1 = mst.common_time(step=0.5).standardize()
ms1.plot(ylabel='SST')

Now let's apply PCA:

In [ ]:
res = ms1.pca()

The result is an object containing various things, and with two plotting method attached to it. For instance, this is how you'd visualize the eigenvalue spectrum.

In [ ]:
res.screeplot()

Eigenvalues themselves are not too informative;  more use is the percentage of variance accounted for by each mode, which is saved as `pctvar`:

In [ ]:
print(res.pctvar.round())

The first mode captures nearly 90% of the common variations between all 4 series. Let's give it its own series and plot it against the original ones:

In [ ]:
mode1 = pyleo.Series(time=ms1.series_list[0].time,value=res.pcs[:,0],label=r'$PC_1$',value_name='SST (normalized)',time_name ='age', time_unit = 'kyr bP')

In [ ]:
import matplotlib.pyplot as plt 
fig, ax = plt.subplots()
ms1.plot(ax=ax,linewidth=0.5)
mode1.plot(color='black',ax=ax)

**Exercise 3.10** Interpret the result. Why do you think this common signal is so flat?  

**Answer 3.10**